# Digital Filter IIR Design

http://scipy.github.io/devdocs/generated/scipy.signal.iirfilter.html<br>

In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import signal

np.random.seed(123)
FLAG_fig = False

In [ ]:
fc_L  = 3.  # カットオフ周波数 [Hz]
fc_H  = 7.  # 上側カットオフ周波数 [Hz]
fsmp  = 50. # サンプリング周波数 [Hz]
fnyq  = fsmp/2.0 # ナイキスト周波数 [Hz]

Ndeg = 6  # フィルタ次数

#### ローパスフィルタ
iirfilterを用いてディジタルフィルタを設計する場合，カットオフ周波数を指定するパラメータWnにはナイキスト周波数で規格化した0～1の値が入る。

In [ ]:
b1, a1  = signal.iirfilter(N=Ndeg, Wn=fc_L/fnyq, 
                           btype='lowpass', analog=False, ftype='butter')
#分子多項式の係数がb1,  分子多項式の係数がa1に代入される
#print(b1)
#print(a1)

#### scipy.signal.freqz
https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqz.html <br>
離散時間伝達関数ゆえ，下記の左辺にあるwには，w = 0 - pi が代入される

In [ ]:
w, h = signal.freqz(b1, a1)
gain = 20*np.log10(abs(h))

In [ ]:
fig, (ax1, ax2)  = plt.subplots(nrows=2, figsize=(6,8))

# Plot Gain
ax1.plot(w, gain, c='b')
ax1.axhline(-3, color='g', ls='--') # -3[dB]
ax1.set_ylim(-60,5)

ax1.grid(which = "both", axis='both', ls="--")
ax1.set_xlabel('omega [radian / sample]')
ax1.set_ylabel('Gain [dB]')

# x-axis [Hz]
f = (w/np.pi)*fnyq

# Gain - Linear
ax2.plot(f, abs(h), c='b')
ax2.axhline(1/np.sqrt(2), color='g', ls='--') # 1/sqrt(2)

ax2.grid(which = "both", axis='both', ls="--")
ax2.set_xticks(np.arange(f.min(),f.max(),1))
ax2.set_xlabel('f [Hz]')
ax2.set_ylabel('Gain [Linear]')


# Phase
ax21 = ax2.twinx()
phase = np.unwrap(np.angle(h))  # radian
#phase = np.unwrap(np.angle(h))*(180./np.pi)  # degree
ax21.plot(f, phase, c='c')
ax21.set_ylabel('Phase [radian]')

if FLAG_fig: plt.savefig('fig_DFIL_IIR_01.png')
plt.show()

#### バンドパスフィルタ

In [ ]:
b2, a2 = signal.iirfilter(N=Ndeg, Wn=[fc_L/(fnyq), fc_H/(fnyq)], 
                          btype='bandpass', analog=False, ftype='butter')
w, h = signal.freqz(b2, a2)
gain = 20*np.log10(abs(h))

In [ ]:
fig, (ax1, ax2)  = plt.subplots(nrows=2, figsize=(6,8))

# Plot Gain
ax1.plot(w, gain, c='b')
ax1.axhline(-3, color='g', ls='--') # -3[dB]
ax1.set_ylim(-60,5)

ax1.grid(which = "both", axis='both', ls="--")
ax1.set_xlabel('omega [radian / sample]')
ax1.set_ylabel('Gain [dB]')

# x-axis [Hz]
f = (w/np.pi)*fnyq

# Gain - Linear
ax2.plot(f, abs(h), c='b')
ax2.axhline(1/np.sqrt(2), color='g', ls='--') # 1/sqrt(2)

ax2.grid(which = "both", axis='both', ls="--")
ax2.set_xticks(np.arange(f.min(),f.max(),1))
ax2.set_xlabel('f [Hz]')
ax2.set_ylabel('Gain [Linear]')


# Phase
ax21 = ax2.twinx()
phase = np.unwrap(np.angle(h))  # radian
#phase = np.unwrap(np.angle(h))*(180./np.pi)  # degree
ax21.plot(f, phase, c='c')
ax21.set_ylabel('Phase [radian]')

if FLAG_fig: plt.savefig('fig_DFIL_IIR_02.png')
plt.show()

# フィルタの効用を見る
scipy.signal.lfilter   https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.lfilter.html

In [ ]:
frq1, frq2 = 1.0, 5.0
Num = 256    # データ数
dt  = 1/fsmp      # サンプリング時間
t = np.linspace(0, Num-1, Num)*dt
y1 = np.sin(2*np.pi*frq1*t)
y2 = np.sin(2*np.pi*frq2*t)
y = y1 + y2 + 0.2*np.random.randn(t.size)

y_filt1 = signal.lfilter(b1,a1,y)

In [ ]:
fig = plt.subplots(figsize=(8,4))

plt.plot(t,y, 'k', label='observed')
plt.plot(t,y_filt1, 'g', label='filtered')
delay = 1.3
plt.plot(t, np.sin(2*np.pi*frq1*t-delay), c='r', ls='--', label='y1 with lag')

plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Low pass')
plt.legend()

if FLAG_fig: plt.savefig('fig_DFIL_IIR_03.png')
plt.show()

In [ ]:
y2 = signal.lfilter(b2,a2,y)

fig = plt.subplots(figsize=(8,4))

#plt.plot(t,y, 'k')
plt.plot(t,y2, 'g', label='filtered')
delay = 1.1
plt.plot(t, np.sin(2*np.pi*frq2*t-delay), c='r', ls='--', label='y2 with lag')

plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Band pass')
plt.legend()

if FLAG_fig: plt.savefig('fig_DFIL_IIR_04.png')
plt.show()